# The Iris Dataset with Graph Execution



## Setup program

### Configure imports



In [0]:
from __future__ import absolute_import, division, print_function

import os
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.ERROR)

## Import and parse the training dataset


### Download the dataset



In [0]:
training_dataset_url = "http://download.tensorflow.org/data/iris_training.csv"
training_dataset_path = tf.keras.utils.get_file(
    fname=os.path.basename(training_dataset_url),
    origin=training_dataset_url)

print("Location of the training dataset file: {}".format(training_dataset_path))

### Inspect the data



In [0]:
!head -n5 {training_dataset_path}

### Parse the dataset


In [0]:
csv_column_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species']

training_dataframe = pd.read_csv(training_dataset_path, names=csv_column_names, header=0)
target_name = 'species'
training_features, training_target = training_dataframe, training_dataframe.pop(target_name)

### Create the training tf.data.Dataset



In [0]:
feature_columns = []
for key in training_features.keys():
    feature_columns.append(tf.feature_column.numeric_column(key=key))

## Select the type of model



### Create a Classifier



In [0]:
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    hidden_units=[10, 10],
    n_classes=3)

## Train the model



### Define the loss and gradient function



In [0]:
def train_input_fn(features, labels, batch_size):
    """An input function for training"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle, repeat, and batch the examples.
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)

    # Return the dataset.
    return dataset

### Create an optimizer



### Training loop



In [0]:
batch_size = 100
steps = 1000
classifier.train(
    input_fn=lambda:train_input_fn(training_features, training_target, batch_size),
    steps=steps)

### Visualize the loss function over time

## Evaluate the model's effectiveness



### Setup the test dataset



In [0]:
test_dataset_url = "http://download.tensorflow.org/data/iris_test.csv"
test_dataset_path = tf.keras.utils.get_file(
    fname=os.path.basename(test_dataset_url),
    origin=test_dataset_url)

print("Location of the test dataset file: {}".format(test_dataset_path))

test = pd.read_csv(test_dataset_path, names=csv_column_names, header=0)
y_name='species'
test_x, test_y = test, test.pop(y_name)


### Evaluate the model on the test dataset



In [0]:
def eval_input_fn(features, labels, batch_size):
    """An input function for evaluation or prediction"""
    features=dict(features)
    if labels is None:
        # No labels, use only features.
        inputs = features
    else:
        inputs = (features, labels)

    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices(inputs)

    # Batch the examples
    assert batch_size is not None, "batch_size must not be None"
    dataset = dataset.batch(batch_size)

    # Return the dataset.
    return dataset

In [0]:
    # Evaluate the model.
    eval_result = classifier.evaluate(
        input_fn=lambda:eval_input_fn(test_x, test_y,
                                                batch_size))

    print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

## Use the trained model to make predictions


In [0]:
SPECIES = ['Iris Setosa', 'Iris Versicolor', 'Iris Virginica']
 
# Generate predictions from the model
expected = ['Setosa', 'Versicolor', 'Virginica']
predict_x = {
        'sepal_length': [5.1, 5.9, 6.9],
        'sepal_width': [3.3, 3.0, 3.1],
        'petal_length': [1.7, 4.2, 5.4],
        'petal_width': [0.5, 1.5, 2.1],
}

predictions = classifier.predict(
        input_fn=lambda:eval_input_fn(predict_x,
                                                labels=None,
                                                batch_size=batch_size))

template = ('\nPrediction is "{}" ({:.1f}%), expected "{}"')

for pred_dict, expec in zip(predictions, expected):
        class_id = pred_dict['class_ids'][0]
        probability = pred_dict['probabilities'][class_id]

        print(template.format(SPECIES[class_id],
                              100 * probability, expec))
